In [1]:
import ccxt
from datetime import datetime
from datetime import timezone as timezone
from dateutil.parser import parse
from time import sleep
import traceback2
import pandas as pd

In [2]:
def __get_client__():

        return ccxt.ftx({
                'headers': {
                    'FTX-SUBACCOUNT': "Tim-Test"},
                    "apiKey":"lUH29pfSzRUumUr1DbhwCWa4XWmGeJK2UK3McjWI",
                    "secret":"6zWU1J69emIgLxtNzEgzNLr7X2brLGsloP0FiCGn"
        })

In [3]:
def get_buy_sell_joined(df):
    buy_df = df[df['side'] == 'buy']
    sell_df = df[df['side'] == 'sell']
    return pd.merge(buy_df, sell_df, how='outer', left_on=['date', 'symbol'], right_on=['date', 'symbol'], suffixes=['_buy', '_sell'])


In [4]:
client = __get_client__()

In [5]:
start_time = datetime(2021, 3, 1).timestamp()
end_time = datetime.now().timestamp()
start_time

1614556800.0

In [6]:
def get_all_orders(start_time, end_time):
    dfs = pd.DataFrame([])
    while True:
        print(start_time, end_time)
        orders = client.private_get_orders_history({'start_time':f"{int(start_time)}", "end_time": f"{int(end_time)}"})['result']
        if len(orders) < 1:
            break
        df = pd.DataFrame(orders)
        df['createdAt'] = pd.to_datetime(df['createdAt'], utc = True)
        min_datetime = df['createdAt'].min()
        print(min_datetime)
        min_datetime = int(min_datetime.timestamp())
        if min_datetime == end_time:
            break
        print(min_datetime)
        dfs = dfs.append(df)
        end_time = min_datetime
    dfs = dfs[dfs['filledSize'] > 0].reset_index(drop=True)
    dfs['date'] = dfs['createdAt'].dt.strftime('%Y-%m-%d')
    dfs = dfs["date,id,market,type,side,price,size,filledSize,avgFillPrice,status,createdAt".split(',')]
    return dfs
        
        

In [29]:
def get_order_aggreated_view(order_df):
    grouped = order_df.groupby(['date', 'market', 'side'])
    agg_df = grouped.agg({'size': ['sum'], "filledSize" : ['sum'], 'price': ['mean'], "avgFillPrice" : ['mean']}).reset_index()
    df = pd.DataFrame([])
    df['date'] = agg_df['date']
    df['symbol'] = agg_df['market']
    df['side'] = agg_df['side']
    df['ordered_size'] = agg_df['size']['sum']
    df['filled_size'] = agg_df['filledSize']['sum']
    df['ordered_price'] = agg_df['price']['mean']
    df['avg_fill_price'] = agg_df['avgFillPrice']['mean']
    df['date'] = pd.to_datetime(df['date'])
    return df.sort_values(['date', 'symbol'])

In [64]:
order_df = get_all_orders(start_time, end_time)
order_agg_df = get_order_aggreated_view(order_df)
daily_orders_df = get_buy_sell_joined(order_agg_df)
# daily_orders_df['pnl'] = (daily_orders_df['filled_size_buy'] * daily_orders_df['avg_fill_price_buy']) - (daily_orders_df['filled_size_sell'] * daily_orders_df['avg_fill_price_buy'])
daily_orders_df.to_csv('daily_orders.csv')


1614556800.0 1625770592.576559
2021-06-19 02:34:08.855129+00:00
1624070048
1614556800.0 1624070048
2021-05-22 17:40:10.494395+00:00
1621705210
1614556800.0 1621705210
2021-04-21 15:41:54.924797+00:00
1619019714
1614556800.0 1619019714
2021-03-17 02:25:26.689566+00:00
1615947926
1614556800.0 1615947926
2021-03-09 15:03:09.399876+00:00
1615302189
1614556800.0 1615302189


In [65]:
daily_orders_df

,date,symbol,side_buy,ordered_size_buy,filled_size_buy,ordered_price_buy,avg_fill_price_buy,side_sell,ordered_size_sell,filled_size_sell,ordered_price_sell,avg_fill_price_sell
0,2021-03-11,BTC-PERP,buy,0.010,0.010,56010.0,56010.0,sell,0.010,0.010,56310.0,56310.000000
1,2021-03-15,BTC-PERP,buy,0.050,0.050,58110.0,58110.0,sell,0.050,0.050,56010.0,56026.000000
2,2021-03-17,BTC-PERP,buy,0.035,0.035,55400.0,55400.0,sell,0.035,0.035,55440.0,55458.500000
3,2021-03-18,BTC-PERP,buy,0.010,0.010,57610.0,57610.0,sell,0.010,0.010,57640.0,57640.000000
4,2021-03-19,BTC-PERP,buy,0.001,0.001,58840.0,58840.0,sell,0.001,0.001,58870.0,58875.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
64,2021-07-07,ETH-PERP,buy,10.000,10.000,2368.5,2368.5,sell,10.000,10.000,2337.0,2337.000000
65,2021-03-29,ETH/USD,NaN,NaN,NaN,NaN,NaN,sell,110.766,110.766,NaN,1804.072651
66,2021-05-23,ETH-PERP,NaN,NaN,NaN,NaN,NaN,sell,1.000,1.000,1785.0,1785.000000
67,2021-05-30,BTC-PERP,NaN,NaN,NaN,NaN,NaN,sell,0.250,0.250,33650.0,33650.000000


In [49]:
def get_all_trades(start_time, end_time):
    dfs = pd.DataFrame([])
    while True:
        print(start_time, end_time)
        trades = client.private_get_fills({'start_time':f"{int(start_time)}", "end_time": f"{int(end_time)}"})['result']
        if len(trades) < 1:
            break
        df = pd.DataFrame(trades)
        df['time'] = pd.to_datetime(df['time'])
        min_datetime = df['time'].min()
        print(min_datetime)
        min_datetime = int(min_datetime.timestamp())
        print(min_datetime)
        df = df[df['type'] != 'otc']
        dfs = dfs.append(df)
        end_time = min_datetime
    dfs['date'] = dfs['time'].dt.strftime('%Y-%m-%d')
    dfs = dfs[dfs['future'].notnull()]
    dfs = dfs["date,id,market,side,price,size,feeRate,fee,time".split(',')]
    return dfs
        
        

In [55]:
def get_trade_aggreated_view(trade_df):
    grouped = trade_df.groupby(['date', 'market', 'side'])
    agg_df = grouped.agg({'size': ['sum'], "feeRate" : ['mean'], 'price': ['mean'], "fee" : ['sum']}).reset_index()
    df = pd.DataFrame([])
    df['date'] = agg_df['date']
    df['symbol'] = agg_df['market']
    df['side'] = agg_df['side']
    df['size'] = agg_df['size']['sum']
    df['price'] = agg_df['price']['mean']
#     df['feeRate'] = agg_df['feeRate']['mean']
    df['fee'] = agg_df['fee']['sum']
    df['date'] = pd.to_datetime(df['date'])
    return df.sort_values(['date', 'symbol'])

In [60]:
trades = get_all_trades(start_time, end_time)
trades_agg_df = get_trade_aggreated_view(trades)
daily_trades_df = get_buy_sell_joined(trades_agg_df)
daily_trades_df['pnl'] = (daily_trades_df['size_sell'] * daily_trades_df['price_sell'] + daily_trades_df['fee_sell']) - (daily_trades_df['size_buy'] * daily_trades_df['price_buy'] + daily_trades_df['fee_buy'])
daily_trades_df.to_csv('daily_trades.csv')

1614556800.0 1625770592.576559
2021-03-03 05:55:55.797152+00:00
1614750955
1614556800.0 1614750955


In [53]:
daily_trades_df

,date,symbol,side_buy,size_buy,price_buy,feeRate_buy,fee_buy,side_sell,size_sell,price_sell,feeRate_sell,fee_sell
0,2021-03-11,BTC-PERP,buy,0.010,56010.000,0.00000,0.000000,sell,0.010,56310.00000,0.000000,0.000000
1,2021-03-15,BTC-PERP,buy,0.050,58110.000,0.00000,0.000000,sell,0.050,56026.00000,0.000340,0.952442
2,2021-03-17,BTC-PERP,buy,0.035,55400.000,0.00000,0.000000,sell,0.035,55458.50000,0.000170,0.475210
3,2021-03-18,BTC-PERP,buy,0.010,57610.000,0.00000,0.000000,sell,0.010,57640.00000,0.000000,0.000000
4,2021-03-19,BTC-PERP,buy,0.001,58840.000,0.00000,0.000000,sell,0.001,58875.00000,0.000340,0.020018
...,...,...,...,...,...,...,...,...,...,...,...,...
59,2021-07-03,ETH-PERP,buy,5.000,2202.000,-0.00001,-0.110100,sell,5.000,2206.00000,-0.000010,-0.110210
60,2021-07-06,AAVE-PERP,buy,10.000,333.000,-0.00001,-0.033300,sell,10.000,334.12500,-0.000010,-0.033525
61,2021-07-06,SNX-PERP,buy,25.000,12.135,-0.00001,-0.003027,sell,25.000,12.15000,-0.000010,-0.003031
62,2021-07-07,ETH-PERP,buy,10.000,2370.000,-0.00001,-0.236850,sell,10.000,2337.00000,-0.000010,-0.233700


In [56]:
daily_trades_df['pnl'] = (daily_trades_df['size_sell'] * daily_trades_df['price_sell'] + daily_trades_df['fee_sell']) - (daily_trades_df['size_buy'] * daily_trades_df['price_buy'] + daily_trades_df['fee_buy'])


In [58]:
daily_trades_df['pnl'].sum()

-3264.6804750465067